In [1]:
from src.sim_utils.sim import Team, GameState
import pandas as pd
from utils.quack import Quack
#import yaml
#import matplotlib.pyplot as plt

C:\Users\Shared.Westover\AppData\Roaming\uv\python\cpython-3.12.5-windows-x86_64-none\Lib\pickle.py:1753: UserWarning: [16:18:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [2]:
schedule = Quack.query('select season, week, away_team, home_team, ifnull(wind, 3) as wind, ifnull(temp,75) as temp, ifnull(spread_line,0) as spread_line from schedule where season = 2024 and week between 0 and 10')
schedule

,season,week,away_team,home_team,wind,temp,spread_line
0,2024,1,BAL,KC,8,67,3.0
1,2024,1,GB,PHI,3,75,2.0
2,2024,1,PIT,ATL,3,75,4.0
3,2024,1,ARI,BUF,20,61,6.5
4,2024,1,TEN,CHI,8,67,4.0
...,...,...,...,...,...,...,...
147,2024,10,TEN,LAC,3,75,8.0
148,2024,10,NYJ,ARI,3,75,-2.0
149,2024,10,PHI,DAL,3,75,-7.5
150,2024,10,DET,HOU,3,75,-4.5


In [3]:
vik = Team("DET", 2025, 4)
det = Team("CHI", 2025, 4)
game = GameState(vik, det, **{"wind": 5, "temp": 90, "spread_line": -12.5})
game.start_game()
game.play_game()
display(game.game_results(df=True).groupby(["team"]).sum()[ 
	["receiving_yards", "rushing_yards"]
])
game.game_results(df=True)[['completions','carries','team']].groupby('team').sum()

D:\Github\NFL_sim\src\sim_utils\team.py:208: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  position_data = self.roster[(self.roster["position"] == position)].fillna(0)
D:\Github\NFL_sim\src\sim_utils\team.py:208: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  position_data = self.roster[(self.roster["position"] == position)].fillna(0)


CHI has won 31 - 27


,receiving_yards,rushing_yards
team,,
CHI,211,85
DET,335,122


,completions,carries
team,,
CHI,21,35
DET,29,35


In [17]:
PLAYERS: pd.DataFrame = Quack.query('''select p.*, adv.stuffed,
mediocre, explosive, huge_play, 
first_down, short_first, epa
from player_weekly_agg as p 
left join rusher_adv as adv
on p.gsis_id = adv.gsis_id
and p.game_id = adv.game_id''')
PLAYERS = pd.concat((PLAYERS, Quack.query('select *, player_name as player_display_name from kicker_ratings')))
PLAYERS

,dense_depth,team,position,gsis_id,game_id,season,week,season_1,player_name,dense_depth_raw,...,mediocre,explosive,huge_play,first_down,short_first,epa,gameday,pos_rank,formation,kicker_rating
0,0,DEN,RB,00-0029683,2013_01_BAL_DEN,2013,1,2013.0,Ronnie,0.0,...,0.266112,0.069383,0.013662,0.206809,0.518358,-0.089818,NaN,NaN,NaN,NaN
1,1,DEN,RB,00-0030522,2013_01_BAL_DEN,2013,1,2013.0,Montee,2.0,...,0.378135,0.072833,0.016618,0.197262,0.662604,-0.053570,NaN,NaN,NaN,NaN
2,2,DEN,RB,00-0026988,2013_01_BAL_DEN,2013,1,2013.0,Knowshon,3.0,...,0.327247,0.062215,0.010865,0.218361,0.894599,-0.047311,NaN,NaN,NaN,NaN
3,0,DET,RB,00-0024217,2013_01_MIN_DET,2013,1,2013.0,Reggie,0.0,...,0.296885,0.121900,0.016297,0.214346,0.607556,-0.036713,NaN,NaN,NaN,NaN
4,1,DET,RB,00-0027218,2013_01_MIN_DET,2013,1,2013.0,Joique,1.0,...,0.318578,0.130564,0.033499,0.220269,0.767657,0.069813,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14950,0,BAL,K,00-0040074,2025_14_PIT_BAL,2025,14,NaN,Tyler Loop,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-07,1,Special Teams,NaN
14951,0,GB,K,00-0029822,2025_14_CHI_GB,2025,14,NaN,Brandon McManus,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-07,1,Special Teams,1691.993020
14952,0,ATL,K,00-0033862,2025_14_SEA_ATL,2025,14,NaN,Zane Gonzalez,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-07,1,Special Teams,1763.657000
14953,0,NYJ,K,00-0025565,2025_14_MIA_NYJ,2025,14,NaN,Nick Folk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-07,1,Special Teams,2056.668302


In [4]:
det.Ks[0].name

'Cairo Santos'

In [16]:
PLAYERS.loc[PLAYERS.team=='DET']['player_display_name']

3             Reggie Bush
4             Joique Bell
44       Matthew Stafford
98            Reggie Bush
99            Joique Bell
               ...       
14806                 NaN
14841                 NaN
14873                 NaN
14891                 NaN
14928                 NaN
Name: player_display_name, Length: 2839, dtype: object

In [8]:
players: pd.DataFrame = Quack.query('''select p.*, adv.stuffed,
mediocre, explosive, huge_play, 
first_down, short_first, epa
from player_weekly_agg as p 
left join rusher_adv as adv
on p.gsis_id = adv.gsis_id
and p.game_id = adv.game_id''')
players.loc[(players.team=='DET')
			& (players.game_id =='2025_07_TB_DET')]

,dense_depth,team,position,gsis_id,game_id,season,week,season_1,player_name,dense_depth_raw,...,yards_per_carry_roll,yards_per_catch_roll,yards_per_target_roll,stuffed,mediocre,explosive,huge_play,first_down,short_first,epa
78284,0,DET,QB,00-0033106,2025_07_TB_DET,2025,7,2025,Jared Goff,0,...,1.793453,4.003168,7.000000,0.244587,0.123346,0.038782,0.004879,0.554896,0.738315,0.236392
79850,0,DET,WR,00-0036963,2025_07_TB_DET,2025,7,2025,Amon-Ra St. Brown,0,...,3.132369,10.983451,9.937801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79851,1,DET,WR,00-0037240,2025_07_TB_DET,2025,7,2025,Jameson Williams,1,...,4.145428,13.528618,8.767536,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79852,2,DET,WR,00-0040669,2025_07_TB_DET,2025,7,2025,Isaac TeSlaa,2,...,3.000000,8.467374,8.404227,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79853,3,DET,WR,00-0032464,2025_07_TB_DET,2025,7,2025,Kalif Raymond,3,...,3.264789,6.540223,6.413402,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79854,4,DET,WR,00-0040241,2025_07_TB_DET,2025,7,2025,Dominic Lovett,4,...,3.000000,4.000000,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79936,2,DET,RB,00-0035567,2025_07_TB_DET,2025,7,2025,Craig Reynolds,2,...,3.435299,5.116717,7.039677,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79937,3,DET,RB,00-0038896,2025_07_TB_DET,2025,7,2025,Jacob Saylors,4,...,3.885609,4.000000,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79997,0,DET,TE,00-0039065,2025_07_TB_DET,2025,7,2025,Sam LaPorta,0,...,3.000060,11.185833,9.692417,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79998,1,DET,TE,00-0036754,2025_07_TB_DET,2025,7,2025,Brock Wright,1,...,3.000000,5.430559,6.010149,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
small = schedule.loc[(schedule.home_team == 'MIA') | (schedule.away_team == 'MIA')]

In [5]:
def log_sim(team1,team2, n=1,**kwargs):
	result = []
	game = GameState(team1, team2, **kwargs)
	for _ in range(n):
		print(_)
		game.start_game()
		game.play_game()
		result += game.game_results()
		game.reset_game()
	return result

def sim_from_schedule(schedule):
	stats = []
	for _, row in schedule.iterrows():
		try:
			team1 = Team(row['home_team'], row['season'], row['week'])
			team2 = Team(row['away_team'], row['season'], row['week'])
			print(team1,team2)
			stats += log_sim(team1,team2, n=1, wind=row['wind'], temp=row['temp'], spread_line=row['spread_line'])
		except Exception as e:
			print(team1.name, team2.name, row['season'],row['week'],e)
	return stats

stats = sim_from_schedule(small)
df = pd.DataFrame(stats)

MIA has 0 points JAX has 0 points
0
JAX has won 24 - 20
MIA has 0 points BUF has 0 points
0
BUF has won 16 - 13
SEA has 0 points MIA has 0 points
0
SEA has won 31 - 17
MIA has 0 points TEN has 0 points
0
MIA has won 34 - 30
NE has 0 points MIA has 0 points
0
MIA has won 21 - 14
IND has 0 points MIA has 0 points
0
MIA has won 26 - 24
No data exists for MIA in 2024
IND MIA 2024 8 list index out of range
BUF has 0 points MIA has 0 points
0
BUF has won 38 - 24
LA has 0 points MIA has 0 points
0
LA has won 31 - 30


In [6]:
szn = Quack.query('select * from weekly where season=2025 and week between 0 and 13')
szn = szn.groupby(['team','week'])[['rushing_yards', 'carries','receptions','receiving_yards', 'passing_yards','completions','attempts']].sum().reset_index().groupby(['team']).mean().round(2).reset_index().sort_values(by='carries', ascending=False)
szn['ypc'] = szn['rushing_yards'] / szn['carries']
szn['yp_pass'] = szn['receiving_yards'] / szn['attempts']
szn['plays'] = szn['attempts'] + szn['carries']
szn.sort_values(by='plays')


,team,week,rushing_yards,carries,receptions,receiving_yards,passing_yards,completions,attempts,ypc,yp_pass,plays
30,TEN,5.60,78.90,20.00,19.40,195.40,195.40,19.40,33.20,3.945000,5.885542,53.20
2,BAL,5.90,142.00,27.40,17.80,199.90,199.90,17.80,26.20,5.182482,7.629771,53.60
26,PIT,6.10,88.20,22.90,20.60,210.80,210.80,20.60,30.70,3.851528,6.866450,53.60
19,MIA,6.00,112.82,23.64,20.45,201.45,201.45,20.45,30.00,4.772420,6.715000,53.64
18,LV,5.80,81.40,22.60,20.70,209.00,209.00,20.70,31.20,3.601770,6.698718,53.80
20,MIN,6.00,103.40,22.20,19.00,213.20,213.20,19.00,31.80,4.657658,6.704403,54.00
24,NYJ,5.70,141.60,28.20,17.20,164.20,164.20,17.20,27.50,5.021277,5.970909,55.70
31,WAS,6.00,138.55,28.00,17.91,204.45,204.45,17.91,28.00,4.948214,7.301786,56.00
25,PHI,5.70,115.20,29.20,18.00,199.50,199.50,18.00,26.90,3.945205,7.416357,56.10
6,CIN,5.60,81.40,19.10,23.60,239.90,239.90,23.60,37.30,4.261780,6.431635,56.40


In [13]:
pd.DataFrame(game.pbp)[['drive','yardline_100']].groupby('drive')['yardline_100'].first()

drive
0     65
1     65
2     79
3     71
4     65
5     65
6     65
7     65
8     82
9     65
10    93
11    50
12    65
13    61
14    65
15    65
16    87
17    65
18    65
19    65
20    91
21    65
22    65
Name: yardline_100, dtype: int64

In [7]:
simszn = (
    df.groupby(["team", "sim_id"])[
        [
            "rushing_yards",
            "carries",
            "receptions",
            "receiving_yards",
            "passing_yards",
            "completions",
            "attempts",
        ]
    ]
    .sum()
    .reset_index()
    .groupby(["team"])
    .mean()
    .round(2)
    .reset_index()
    .sort_values(by="carries", ascending=False)
) #group by sim first to get team stats for each game, then group by team to get per game avg

simszn["ypc"] = simszn["rushing_yards"] / simszn["carries"]
simszn["yp_pass"] = simszn["receiving_yards"] / simszn["attempts"]
simszn["plays"] = simszn["attempts"] + simszn["carries"]
simszn.sort_values(by="yp_pass")

,team,sim_id,rushing_yards,carries,receptions,receiving_yards,passing_yards,completions,attempts,ypc,yp_pass,plays
5,NE,1.764360e+12,79.00,20.00,17.00,175.00,175.00,17.00,34.00,3.950000,5.147059,54.00
1,IND,1.764360e+12,194.00,38.00,21.00,240.00,240.00,21.00,42.00,5.105263,5.714286,80.00
3,LA,1.764360e+12,65.00,24.00,18.00,202.00,202.00,18.00,34.00,2.708333,5.941176,58.00
4,MIA,1.764360e+12,108.88,28.38,23.75,267.38,267.38,23.75,36.88,3.836505,7.250000,65.26
7,TEN,1.764360e+12,66.00,31.00,33.00,370.00,370.00,33.00,44.00,2.129032,8.409091,75.00
0,BUF,1.764360e+12,91.00,30.00,27.50,350.50,350.50,27.50,38.50,3.033333,9.103896,68.50
6,SEA,1.764360e+12,99.00,31.00,36.00,367.00,367.00,36.00,39.00,3.193548,9.410256,70.00
2,JAX,1.764360e+12,57.00,29.00,26.00,358.00,358.00,26.00,36.00,1.965517,9.944444,65.00


In [57]:
(
	df.groupby(['name', 'team'])[['receptions','carries','receiving_yards','rushing_yards', 'passing_yards']]
	.median()
	.round(2)
	.reset_index()
).sort_values(by='passing_yards')


,name,team,receptions,carries,receiving_yards,rushing_yards,passing_yards
0,AJ Barner,SEA,0.0,0.0,0.0,0.0,0.0
1,Adonai Mitchell,IND,1.0,0.0,23.0,0.0,0.0
2,Alec Pierce,IND,3.0,0.0,58.0,0.0,0.0
3,Andrew Ogletree,IND,0.0,0.0,0.0,0.0,0.0
5,Antonio Gibson,NE,2.0,8.0,17.0,36.0,0.0
...,...,...,...,...,...,...,...
94,Tua Tagovailoa,MIA,0.0,3.0,0.0,8.5,310.0
4,Anthony Richardson,IND,0.0,12.0,0.0,59.0,319.0
91,Trevor Lawrence,JAX,0.0,7.0,0.0,12.0,328.0
35,Geno Smith,SEA,0.0,7.0,0.0,19.0,348.0


In [80]:
Quack.query("""
			select player_display_name as name, team, median(carries), median(receptions)
        from weekly
        where season = 2025 
        group by all order by 3 desc
			""")

,name,team,median(carries),median(receptions)
0,Saquon Barkley,PHI,18.0,3.0
1,Derrick Henry,BAL,18.0,1.0
2,Quinshon Judkins,CLE,18.0,1.0
3,Josh Jacobs,GB,17.5,3.5
4,James Cook,BUF,17.0,3.0
...,...,...,...,...
1876,Josh Whyle,GB,0.0,1.0
1877,Younghoe Koo,ATL,0.0,0.0
1878,Truman Jones,TEN,0.0,0.0
1879,Cameron Johnston,BUF,0.0,0.0


In [83]:
def compare_stats(df: pd.DataFrame, stats: list, season: int=2025, start_week: int=0, end_week: int=13, min_filter=None):
    """
    Query real avg stats from DB, compute median predicted stats, and compare them.
    
    quack: DB connection with .query()
    df: DataFrame with per-game predicted stats (includes 'name','team', and stat columns)
    stats: list of stat names (e.g. ['receptions','receiving_yards'])
    min_filter: optional dict to filter rows before mean, e.g. {'receptions': 2}
    """
    # --- build SQL dynamically ---
    select_cols = ", ".join([f"median({s}) as {s}" for s in stats])
    sql = f"""
        select player_display_name as name, team, {select_cols}
        from weekly
        where season = {season} and week between {start_week} and {end_week}
        group by all
    """
    real = Quack.query(sql)

    # --- median predicted stats ---
    med = (
        df.groupby(['name', 'team'])[stats]
        .median()
        .round(2)
        .reset_index()
    )

    # --- merge + compare ---
    comb = real.merge(med, on=['name', 'team'], suffixes=('_real', '_pred'))
    col_order = ['name','team'] + [f"{s}_{suffix}" for s in stats for suffix in ["pred", "real"]]
    comb = comb[col_order]
    return comb
    results = {}
    for s in stats:
        res_col = f"{s}_res"
        comb[res_col] = comb[f"{s}_real"] > comb[f"{s}_pred"]
        tmp = comb
        if min_filter and s in min_filter:
            tmp = tmp[tmp[f"{s}_pred"] > min_filter[s]]
        results[s] = tmp[res_col].mean().round(2)
    return results

comp = compare_stats(df, ['receptions','carries','receiving_yards','rushing_yards', 'passing_yards'], min_filter=[2,2,20,20,40]).sort_values(by='rushing_yards_real', ascending=False)
comp

,name,team,receptions_pred,receptions_real,carries_pred,carries_real,receiving_yards_pred,receiving_yards_real,rushing_yards_pred,rushing_yards_real,passing_yards_pred,passing_yards_real
43,James Cook,BUF,4.0,3.0,10.5,17.0,62.0,11.0,30.5,108.0,0.0,0.0
22,Kyren Williams,LA,1.0,2.0,18.0,14.0,10.0,12.5,48.0,69.5,0.0,0.0
18,De'Von Achane,MIA,4.5,5.0,12.5,14.0,37.0,29.0,26.0,67.0,0.0,0.0
7,Travis Etienne,JAX,5.0,2.0,9.0,16.0,70.0,12.0,30.0,64.5,0.0,0.0
28,Tony Pollard,TEN,3.0,2.0,16.0,12.5,22.0,12.0,25.0,50.5,0.0,0.0
45,Zach Charbonnet,SEA,4.0,1.0,3.0,12.0,27.0,6.0,12.0,37.0,0.0,0.0
32,Josh Allen,BUF,0.0,0.0,9.0,6.0,0.0,0.0,24.5,31.0,396.0,253.0
27,Blake Corum,LA,1.0,0.0,7.0,8.0,5.0,0.0,27.0,30.0,0.0,0.0
46,Antonio Gibson,NE,2.0,0.0,8.0,6.0,17.0,0.0,36.0,27.0,0.0,0.0
11,Rhamondre Stevenson,NE,3.0,2.0,11.0,10.0,10.0,12.5,33.0,26.0,0.0,0.0


In [49]:
df = game.game_results(df=True)
df[['sim_id','team', 'position', 'name',
	'carries', 'rushing_yards',       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
	'rushing_tds', 
	'rushing_first_downs',  'rushing_2pt_conversions',
	'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
	'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
	'receiving_2pt_conversions', 'target_share', 'air_yards_share',
	'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',
	'air_yards', 'yac']].sum()

sim_id                                                            37047459497832
team                           CHICHICHICHICHICHICHICHICHICHICHIDETDETDETDETD...
position                              QBQBRBRBWRWRWRWRTETETEQBRBRBRBRBWRWRWRTETE
name                           Caleb WilliamsTyson BagentD'Andre SwiftTravis ...
carries                                                                       50
rushing_yards                                                                253
receptions                                                                    56
targets                                                                       84
receiving_yards                                                              672
receiving_tds                                                                  0
rushing_tds                                                                    0
rushing_first_downs                                                            0
rushing_2pt_conversions     

In [93]:
for stat in test:
	comp[stat + '_diff'] = comp[stat + '_real'] - comp[stat + '_pred']
diffs = [x + '_diff' for x in test]
comp['receptions_diff'].sort_values().reset_index()

,index,receptions_diff
0,36,-5.0
1,13,-4.0
2,37,-4.0
3,19,-4.0
4,23,-3.5
5,7,-3.0
6,50,-3.0
7,45,-3.0
8,21,-2.0
9,38,-2.0


In [50]:
df = (
	game.game_results(df=True)
	.groupby(["team"])
	.sum()[["receiving_yards", "rushing_yards", "carries"]]
)
df["ypc"] = df["rushing_yards"] / df["carries"]
df

,receiving_yards,rushing_yards,carries,ypc
team,,,,
CHI,393,87,24,3.625000
DET,279,166,26,6.384615


In [ ]:
def run_sim(team1, team2, num_sims, config=config):
	results = []
	scores = []
	vik = Team(team1, 2024, 13)
	det = Team(team2, 2024, 13)
	game = GameState(vik, det, config, **{"wind": 5, "temp": 60, "spread_line": 6.5})
	for i in range(num_sims):
		game.start_game()
		game.play_game()
		scores.append((game.home.score, game.away.score))
		res = vik.game_results() + det.game_results()
		for result in res:
			result["game_number"] = i
		results += res
	return results, scores


results, scores = run_sim(
	"DET", "CHI", 10
)  # 28.8 -> 22.8 by pulling game initialization out of loop

DET has won 20 - 13
CHI has won 35 - 21
DET has won 35 - 20
CHI has won 28 - 20
CHI has won 35 - 24
DET has won 14 - 9
CHI has won 24 - 20
DET has won 27 - 14
DET has won 28 - 21
DET has won 42 - 20


In [ ]:
df = pd.json_normalize(results)
df.groupby(["name", "id"])[
	[
		"receiving_yards",
		"targets",
		"receptions",
		"air_yards",
		"completions",
		"attempts",
		"rushing_yards",
		"carries",
	]
].agg(["median", "mean"]).sort_values(by=[("receiving_yards", "mean")], ascending=False)

receiving_yards       targets      receptions  \
                                      median  mean  median mean     median   
name              id                                                         
Sam LaPorta       00-0039065            74.5  88.4     8.5  8.9        5.5   
Amon-Ra St. Brown 00-0036963            86.0  87.0     8.0  7.9        4.5   
Jahmyr Gibbs      00-0039139            66.0  78.4     7.0  6.8        4.5   
Cole Kmet         00-0036290            38.5  60.6     5.0  5.3        2.5   
Rome Odunze       00-0039919            52.5  56.9     4.5  4.9        2.5   
D.J. Moore        00-0034827            40.5  50.5     6.5  6.7        3.0   
Keenan Allen      00-0030279            40.0  40.4     5.5  6.5        3.0   
Tim Patrick       00-0033375            15.5  32.3     3.0  3.1        1.5   
D'Andre Swift     00-0036275            30.0  30.3     4.0  4.2        2.5   
David Montgomery  00-0035685            19.5  29.4     4.5  4.2        3.0   
Allen Robinson    00-0031428             0.5  17.0     1.0  1.3        0.5   
Marcedes Lewis    00-0024243             1.5   9.9     0.5  0.8        0.5   
Brock Wright      00-0036754             0.0   9.4     1.5  1.4        1.0   
Roschon Johnson   00-0039021             0.0   8.7     1.0  1.2        0.0   
Gerald Everett    00-0033895             0.0   6.2     0.0  0.4        0.0   
DeAndre Carter    00-0031763             0.0   0.8     0.5  0.6        0.0   
Caleb Williams    00-0039918             0.0   0.0     0.0  0.0        0.0   
Jared Goff        00-0033106             0.0   0.0     0.0  0.0        0.0   
Craig Reynolds    00-0035567             0.0   0.0     0.0  0.0        0.0   
Sione Vaki        00-0039364             0.0   0.0     0.0  0.0        0.0   
Travis Homer      00-0035594             0.0   0.0     0.0  0.0        0.0   

                                  air_yards       completions       attempts  \
                             mean    median  mean      median  mean   median   
name              id                                                           
Sam LaPorta       00-0039065  5.5      38.0  49.0         0.0   0.0      0.0   
Amon-Ra St. Brown 00-0036963  4.4      30.0  44.1         0.0   0.0      0.0   
Jahmyr Gibbs      00-0039139  4.6       9.5  15.3         0.0   0.0      0.0   
Cole Kmet         00-0036290  3.1      18.5  35.0         0.0   0.0      0.0   
Rome Odunze       00-0039919  3.0      23.0  36.3         0.0   0.0      0.0   
D.J. Moore        00-0034827  3.3       8.0  11.8         0.0   0.0      0.0   
Keenan Allen      00-0030279  2.9       5.0  18.6         0.0   0.0      0.0   
Tim Patrick       00-0033375  1.5       4.0  17.1         0.0   0.0      0.0   
D'Andre Swift     00-0036275  2.4       7.5  11.0         0.0   0.0      0.0   
David Montgomery  00-0035685  2.7      -3.5  -0.8         0.0   0.0      0.0   
Allen Robinson    00-0031428  0.8       0.0   7.4         0.0   0.0      0.0   
Marcedes Lewis    00-0024243  0.6       0.0   2.0         0.0   0.0      0.0   
Brock Wright      00-0036754  0.7       0.0   1.8         0.0   0.0      0.0   
Roschon Johnson   00-0039021  0.5       0.0   0.4         0.0   0.0      0.0   
Gerald Everett    00-0033895  0.4       0.0   4.4         0.0   0.0      0.0   
DeAndre Carter    00-0031763  0.2       0.0   0.2         0.0   0.0      0.0   
Caleb Williams    00-0039918  0.0       0.0   0.0        16.0  16.4     30.5   
Jared Goff        00-0033106  0.0       0.0   0.0        18.0  20.2     32.5   
Craig Reynolds    00-0035567  0.0       0.0   0.0         0.0   0.0      0.0   
Sione Vaki        00-0039364  0.0       0.0   0.0         0.0   0.0      0.0   
Travis Homer      00-0035594  0.0       0.0   0.0         0.0   0.0      0.0   

                                   rushing_yards       carries        
                              mean        median  mean  median  mean  
name              id                                                  
Sam LaPorta       00-0

In [ ]:
df.groupby(["team_name", "game_number"])[
	[
		"receiving_yards",
		"targets",
		"receptions",
		"air_yards",
		"completions",
		"attempts",
		"rushing_yards",
		"carries",
	]
].sum().groupby(["team_name"]).mean().sort_values(
	by=["receiving_yards"], ascending=False
)

,receiving_yards,targets,receptions,air_yards,completions,attempts,rushing_yards,carries
team_name,,,,,,,,
CAR,393.4,34.9,22.3,190.7,22.3,34.9,142.9,27.7
PHI,340.0,33.2,19.7,177.1,19.7,33.2,164.4,35.5


In [ ]:
sched = pd.read_csv("week_13.csv", index_col=0)
sched.iloc[11]

game_id           2024_13_TB_CAR
season                      2024
week                          13
home_team                    CAR
away_team                     TB
home_moneyline             205.0
spread_line                 -6.5
total_line                  46.5
temp                        50.0
wind                         8.0
Name: 6896, dtype: object

In [ ]:
def run_sim(num_sims, sched, config=config):
	results = []
	scores = []
	for i in range(num_sims):
		vik = Team(sched["home_team"], sched["season"], sched["week"])
		det = Team(sched["away_team"], sched["season"], sched["week"])
		game = GameState(vik, det, config, **sched)
		game.start_game()
		game.play_game()
		scores.append((game.home.score, game.away.score))
		res = vik.game_results() + det.game_results()
		for result in res:
			result["game_number"] = i
		results += res
	return results, scores


# results, scores = run_sim('KC','LV',100)

In [ ]:
results = []
test = True
if test:
	for t in [11]:
		# for t in range(len(sched)):
		game1 = sched.iloc[t].to_dict()
		try:
			results, scores = run_sim("KC", "LV", 100, game1)
		except Exception as _:
			print("oops")

CAR has won 13 - 7
CAR has won 31 - 7
CAR has won 30 - 6
CAR has won 26 - 17
TB has won 28 - 24
CAR has won 34 - 27
TB has won 24 - 14
TB has won 27 - 20
TB has won 28 - 24
CAR has won 24 - 20
CAR has won 17 - 13
CAR has won 21 - 13
CAR has won 40 - 14
CAR has won 24 - 14
TB has won 16 - 16
TB has won 20 - 19
TB has won 23 - 21
CAR has won 26 - 17
TB has won 21 - 17
TB has won 26 - 17
TB has won 17 - 17
CAR has won 13 - 10
CAR has won 24 - 9
CAR has won 45 - 17
CAR has won 44 - 17
TB has won 31 - 23
CAR has won 31 - 21
TB has won 20 - 7
TB has won 17 - 17
CAR has won 23 - 17
TB has won 27 - 6
CAR has won 28 - 27
TB has won 21 - 16
TB has won 24 - 6
TB has won 24 - 14
CAR has won 41 - 16
CAR has won 23 - 17
CAR has won 41 - 38
TB has won 31 - 14
TB has won 20 - 17
TB has won 34 - 14
CAR has won 20 - 10
TB has won 10 - 10
TB has won 31 - 10
CAR has won 34 - 17
TB has won 24 - 23
TB has won 21 - 7
TB has won 26 - 26
CAR has won 41 - 16
CAR has won 20 - 9
TB has won 24 - 24
CAR has won 24 

In [ ]:
weekly = pd.read_csv("data/weekly.csv", index_col=0)
pit = weekly.loc[
	(weekly.week == 12)
	& (weekly.season == 2024)
	& ((weekly.recent_team == "PIT") | (weekly.recent_team == "CLE"))
]